In [ ]:
from code.wrapper import utils

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

**Normalization** increases the speed of calculation and reduces the alpha value while also increasing the score
https://stats.stackexchange.com/a/189179

In [ ]:
from sklearn.preprocessing import RobustScaler, Normalizer
# https://stackoverflow.com/a/23835410

excel_sheet = pd.read_excel("../Data/New/unfiltered_data.xlsx", sheet_name=["full_train", "full_test",
                                                                               "ionizable_train", "ionizable_test",
                                                                               "neutral_train", "neutral_test"])


full_train: pd.DataFrame = excel_sheet["full_train"]
full_test: pd.DataFrame = excel_sheet["full_test"]

neutral_train: pd.DataFrame = excel_sheet["neutral_train"]
neutral_test: pd.DataFrame = excel_sheet["neutral_test"]

ionizable_train: pd.DataFrame = excel_sheet["ionizable_train"]
ionizable_test: pd.DataFrame = excel_sheet["ionizable_test"]

Scaler = RobustScaler()
Norm = Normalizer()
# TRAIN
X_full_train = full_train.loc[:, full_train.columns != "Log_MP_RATIO"]
y_full_train = full_train["Log_MP_RATIO"]


X_neutral_train = neutral_train.loc[:, neutral_train.columns != "Log_MP_RATIO"]
y_neutral_train = neutral_train["Log_MP_RATIO"]

X_ionizable_train = ionizable_train.loc[:, ionizable_train.columns != "Log_MP_RATIO"]
y_ionizable_train = ionizable_train["Log_MP_RATIO"]
# Scaler.fit(X_full_train)
# X_full_train = pd.DataFrame(Scaler.transform(X_full_train), columns = X_full_train.columns)
#
# Norm.fit(X_full_train)
# X_full_train = pd.DataFrame(Norm.transform(X_full_train), columns=X_full_train.columns)

# TEST
X_full_test = full_test.loc[:, full_test.columns != "Log_MP_RATIO"]
y_full_test = full_test["Log_MP_RATIO"]

X_neutral_test = neutral_test.loc[:, neutral_test.columns != "Log_MP_RATIO"]
y_neutral_test = neutral_test["Log_MP_RATIO"]

X_ionizable_test = ionizable_test.loc[:, ionizable_test.columns != "Log_MP_RATIO"]
y_ionizable_test = ionizable_test["Log_MP_RATIO"]
#
# Scaler.fit(X_full_test)
# X_full_test = pd.DataFrame(Scaler.transform(X_full_test), columns = X_full_test.columns)
#
# Norm.fit(X_full_test)
# X_full_test = pd.DataFrame(Norm.transform(X_full_test), columns=X_full_test.columns)

# Full

In [ ]:
test_utils = utils.Utils(full_train)
test_utils.create_cv_folds(display=True)
test_utils.display_score(Ridge(), X_full_train, y_full_train, X_full_test, y_full_test)

In [ ]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-10, 1e10, log=True)

    solver = trial.suggest_categorical('solver', ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"])

    clf = Ridge(max_iter=100000, alpha=alpha, solver=solver)

    # n_ft = trial.suggest_int('n_ft', 1, 10, log=True)
    # clf = RFE(Ridge(max_iter=100000, alpha=alpha, solver=solver), n_features_to_select=n_ft)

    estimator = utils.Utils(full_train)
    # return cross_val_score(clf, X_full_train, y_full_train, cv=5, n_jobs=-1).mean()
    return estimator.cross_value_score(clf)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)
trial = study.best_trial
print(trial.value, trial.params)

In [ ]:
test_utils.display_score(Ridge(**{'alpha': 117191.74850379501, 'solver': 'lsqr'}), X_full_train, y_full_train, X_full_test, y_full_test)
# display(plot_optimization_history(study))

rr = Ridge(**{'alpha': 117191.74850379501, 'solver': 'lsqr'}).fit(X_full_train, y_full_train)
y_full_train_pred = rr.predict(X_full_train)
y_full_test_pred = rr.predict(X_full_test)

test_utils.display_graph(rr, X_full_train, X_full_test, y_full_train, y_full_test)

In [ ]:
rr = Ridge(max_iter=100000, **({'alpha': 117191.74850379501, 'solver': 'lsqr'}))
rr.fit(X_full_train, y_full_train)


hyper_params = [{"n_features_to_select": list(range(1,len(X_full_train.columns)))}]

rfe = RFE(rr, step=1)

model_cv = GridSearchCV(estimator=rfe,
                        param_grid=hyper_params,
                        scoring="r2",
                        cv=3,
                        verbose=2,
                        return_train_score=True,
                        n_jobs=-1)


model_cv.fit(X_full_train, y_full_train)

cv_results = pd.DataFrame(model_cv.cv_results_)

best_rfe = model_cv.best_estimator_


plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel("number of features")
plt.ylabel("R2")
plt.title("Optimal number of features")
plt.legend(["test score", "train score"], loc="upper left")
plt.show()


test_utils.display_score(best_rfe,
                         X_full_train, y_full_train,
                         X_full_test, y_full_test)

print("Selected features:", best_rfe.get_feature_names_out())


In [ ]:
test_utils.display_score(best_rfe, X_full_train, y_full_train, X_full_test, y_full_test)
# display(plot_optimization_history(study))

rr = best_rfe.fit(X_full_train, y_full_train)
y_full_train_pred = rr.predict(X_full_train)
y_full_test_pred = rr.predict(X_full_test)

test_utils.display_graph(rr, X_full_train, X_full_test, y_full_train, y_full_test)

# Ionizable

In [ ]:
test_utils = utils.Utils(ionizable_train)
test_utils.create_cv_folds(display=True)
test_utils.display_score(Ridge(), X_ionizable_train, y_ionizable_train, X_ionizable_test, y_ionizable_test)

In [ ]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-10, 1e10, log=True)

    solver = trial.suggest_categorical('solver', ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"])

    clf = Ridge(max_iter=100000, alpha=alpha, solver=solver)

    # n_ft = trial.suggest_int('n_ft', 1, 10, log=True)
    # clf = RFE(Ridge(max_iter=100000, alpha=alpha, solver=solver), n_features_to_select=n_ft)

    estimator = utils.Utils(ionizable_train)
    # return cross_val_score(clf, X_ionizable_train, y_ionizable_train, cv=5, n_jobs=-1).mean()
    return estimator.cross_value_score(clf)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)
trial = study.best_trial
print(trial.value, trial.params)

In [ ]:
rr = Ridge(max_iter=100000, **({'alpha': 23051583.217470333, 'solver': 'sparse_cg'}))
rr.fit(X_ionizable_train, y_ionizable_train)


hyper_params = [{"n_features_to_select": list(range(1,len(X_ionizable_train.columns)))}]

rfe = RFE(rr, step=1)

model_cv = GridSearchCV(estimator=rfe,
                        param_grid=hyper_params,
                        scoring="r2",
                        cv=3,
                        verbose=2,
                        return_train_score=True,
                        n_jobs=-1)


model_cv.fit(X_ionizable_train, y_ionizable_train)

cv_results = pd.DataFrame(model_cv.cv_results_)

best_rfe = model_cv.best_estimator_


plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel("number of features")
plt.ylabel("R2")
plt.title("Optimal number of features")
plt.legend(["test score", "train score"], loc="upper left")
plt.show()


test_utils.display_score(best_rfe,
                         X_ionizable_train, y_ionizable_train,
                         X_ionizable_test, y_ionizable_test)

print("Selected features:", best_rfe.get_feature_names_out())

In [ ]:

# display(plot_optimization_history(study))

rr = Ridge(**({'alpha': 23051583.217470333, 'solver': 'sparse_cg'})).fit(X_ionizable_train, y_ionizable_train)
y_ionizable_train_pred = rr.predict(X_ionizable_train)
y_ionizable_test_pred = rr.predict(X_ionizable_test)

test_utils.display_graph(rr, X_ionizable_train, X_ionizable_test, y_ionizable_train, y_ionizable_test)

# Neutral

In [ ]:
test_utils = utils.Utils(neutral_train)
test_utils.create_cv_folds(display=True)
test_utils.display_score(Ridge(), X_neutral_train, y_neutral_train, X_neutral_test, y_neutral_test)

In [ ]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-10, 1e10, log=True)

    solver = trial.suggest_categorical('solver', ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"])

    clf = Ridge(max_iter=100000, alpha=alpha, solver=solver)

    # n_ft = trial.suggest_int('n_ft', 1, 10, log=True)
    # clf = RFE(Ridge(max_iter=100000, alpha=alpha, solver=solver), n_features_to_select=n_ft)

    estimator = utils.Utils(neutral_train)
    # return cross_val_score(clf, X_neutral_train, y_neutral_train, cv=5, n_jobs=-1).mean()
    return estimator.cross_value_score(clf)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)
trial = study.best_trial
print(trial.value, trial.params)

In [ ]:
rr = Ridge(max_iter=100000, **{'alpha': 19553992.061135184, 'solver': 'lsqr'})
rr.fit(X_neutral_train, y_neutral_train)


hyper_params = [{"n_features_to_select": list(range(1,len(X_neutral_train.columns)))}]

rfe = RFE(rr, step=1)

model_cv = GridSearchCV(estimator=rfe,
                        param_grid=hyper_params,
                        scoring="r2",
                        cv=3,
                        verbose=2,
                        return_train_score=True,
                        n_jobs=-1)


model_cv.fit(X_neutral_train, y_neutral_train)

cv_results = pd.DataFrame(model_cv.cv_results_)

best_rfe = model_cv.best_estimator_


plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel("number of features")
plt.ylabel("R2")
plt.title("Optimal number of features")
plt.legend(["cross-value score", "train score"], loc="upper left")
plt.show()


test_utils.display_score(best_rfe,
                         X_neutral_train, y_neutral_train,
                         X_neutral_test, y_neutral_test)


print("Selected features:", best_rfe.get_feature_names_out())

In [ ]:

# display(plot_optimization_history(study))

rr = Ridge(**{'alpha': 19553992.061135184, 'solver': 'lsqr'}).fit(X_full_train, y_full_train)
y_full_train_pred = rr.predict(X_full_train)
y_full_test_pred = rr.predict(X_full_test)

test_utils.display_graph(rr, X_full_train, X_full_test, y_full_train, y_full_test)

# Demo of good R2 Q2 but bad cross val

In [ ]:
from sklearn.model_selection import LeaveOneOut, ShuffleSplit, StratifiedShuffleSplit

# THIS IS AN EXEMPLE OF A "GOOD" R2 and Q2 but it won't be found because the cross val is low



rr = Ridge()
rr.fit(X_full_train, y_full_train)

rfe = RFE(rr, n_features_to_select=20)
rfe = rfe.fit(X_full_train, y_full_train)


print(rfe.score(X_full_train, y_full_train))
print(rfe.score(X_full_test, y_full_test))
print(cross_val_score(rfe, X_full_train, y_full_train, scoring="r2", cv=ShuffleSplit(n_splits=4,test_size=0.1,random_state=0)).mean())
